# Setting up

In [ ]:
# All the needed imports
# import tenserflow
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import spikingjelly.activation_based.encoding as encoding
import spikingjelly.activation_based.neuron as neuron
import spikingjelly.activation_based.functional as functional
import matplotlib.pyplot as plt
import numpy as np
import pathlib
import os

**prepocess images**

In [9]:
# Finding the data in the dictionary
data_dir = pathlib.Path("..") / "brain-tumor-mri-dataset"
train_dir = data_dir / "Training"
test_dir = data_dir / "Testing"


transform_train = transforms.Compose([
    transforms.Grayscale(),  # Convert to grayscale
    transforms.Resize((64, 64)),  # Resize for SNN, 64 for saving memory
    transforms.RandomRotation(30),  # Rotation augmentation (randomly rotates the image up to ±30 degrees)
    transforms.RandomHorizontalFlip(), # Flips images horizontally (left to right)
    transforms.ToTensor(),  # Convert to Tensor
    transforms.Normalize((0.5,), (0.5,))  # Normalize, by reducing sharp spikes in activation
])

# Evaluate the model on real, unaltered images, so we dont do image augmantation.
transform_test = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load images to train and test sets
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform_train)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform_test)

# Group multiple images into batches 
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Test correctness
print("Class Labels:", train_dataset.classes)


Class Labels: ['glioma', 'meningioma', 'notumor', 'pituitary']


**poisson encoding**

In [ ]:
# Encoding transforms each pixel into a series of spikes:
# - Poisson encoding converts pixel intensity into a probability of firing.
# - Bright pixels fire more spikes, while dark pixels fire fewer.

# Convert image tensors into spikes
encoder = encoding.PoissonEncoder()

# Define timesteps (how many times an image generates spikes)
timesteps = 10 

# Convert image batch into spike that trains over multiple timesteps.
def encode_batch(batch, timesteps):
    encoded_spikes = torch.stack([encoder(batch) for _ in range(timesteps)], dim=0)  
    return encoded_spikes

# Encode a sample batch
sample_batch, sample_labels = next(iter(train_loader))
spike_batch = encode_batch(sample_batch, timesteps)

print("Original Shape:", sample_batch.shape) # test them
print("Spike Encoded Shape:", spike_batch.shape)


Original Shape: torch.Size([32, 1, 64, 64])
Spike Encoded Shape: torch.Size([10, 32, 1, 64, 64])
